In [1]:
import requests

url = 'https://fbref.com/en/comps/9/stats/Premier-League-Stats'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

response = requests.get(url, headers=headers)

with open("fbref_page.html", "w", encoding="utf-8") as file:
    file.write(response.text)

# This is extracting the raw html from the page into a local HTML file


ModuleNotFoundError: No module named 'requests'

In [45]:
from bs4 import BeautifulSoup
import pandas as pd

# Path to your saved HTML file
html_file = "fbref_page.html"

# Load the HTML file with BeautifulSoup
with open(html_file, "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "lxml")  # You can also try 'lxml' if needed

# Locate the <table> with id 'stats_standard'
table = soup.find("table", {"id": "stats_standard"})

if table:
    print("Found <table id='stats_standard'>.")
    # Print the first 500 characters of the table for inspection
    print("Preview of <table> content:\n", table.prettify()[:500])

    # Parse the table using pandas
    try:
        df = pd.read_html(str(table))[0]
        # Save the DataFrame to a CSV file
        df.to_csv("prem_player_stats.csv", index=False)
        print("Player stats saved to 'prem_player_stats.csv'")
    except ValueError as e:
        print(f"Error parsing the table: {e}")
else:
    print("No <table> with id 'stats_standard' found.")

# This code was obtained from ChatGPT and it is used to extract tables from HTML files using BeautifulSoup and pandas.


Found <table id='stats_standard'>.
Preview of <table> content:
 <table class="min_width sortable stats_table min_width shade_zero" data-cols-to-freeze=",2" data-non-qual="1" data-qual-label="Hide non-qualifiers for rate stats" data-qual-text="" id="stats_standard">
 <caption>
  Player Standard Stats
  <span style="color: #666; font-size:smaller">
   2024-2025 Premier League
  </span>
  Table
 </caption>
 <colgroup>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
 
Player stats saved to 'prem_player_stats.csv'


/var/folders/jp/fbwfkpy50vs_7m6dlkzl468h0000gn/T/ipykernel_63832/2486324339.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


# Explanation Of Code

I had a lot of difficulty extracting this table from the initial code that was on the site, my original plan was to take it directly from the site but my code couldn't pick up the table that I needed to extract for my statistics. So then I tried extracting it to a local HTML file and this is where I started to find out more. I then began trying to extract my desired table from the HTML file that I had created but once again this was unable to pick up the table that I needed, I began reading through the raw HTML code of the site near where my desired table was when I eventually spotted this sign '<--' 2 lines above the table. This got me wondering as I recognised it as a comment symbol and I googled it and realised that comments aren't picked up in the search method I was using so I deleted this line in the local HTML file and ran my code again and the data extracted into my CSV so that I can pre proccess it and begin to build my model.

In [5]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

df = pd.read_csv("prem_player_stats.csv")

# For the example I am going to do I am going to just use attackers

fw_df = df[df["Pos"].str.startswith("FW", na=False)] # I am doing this as some players have multiple positions but their main one is always listed first so I can just use the first two characters to filter

fw_df.to_csv("prem_fw_stats.csv", index=False)
fw_df = pd.read_csv("prem_fw_stats.csv")

This code block is just handling all of my imports for this model and then extracting all the forwards data from my playerbase csv so that I can begin to build my model from this data

In [8]:
filtered_data = fw_df.drop(['Rk', 'Nation', 'Pos', 'Squad', 'Age', 'Born', 'Matches'], axis=1)
filtered_data.to_csv("prem_fw_stats_filtered.csv", index=False)

I am now running into the issue with this data as I'm really not sure as to what my target value in this data is, I would ideally like to create a new field to be the target field but I'm not sure as to what formula I could use to calculate something that would act as a player score as such, I am currently researching the concept of pairwise comparison as I believe that might be useful